In [1]:
import numpy as np
from definitions import path_join, make_directory, EXPERIMENTS_DIR, VENSIM_MODELS_DIR, logging

In [2]:
KNOWN_MODEL = 'known model'
UNKNOWN_MODEL = 'unknown model'

In [3]:
model_name = 'teacup'
experiment_name = '{}_recovery'.format(model_name)

mode = UNKNOWN_MODEL
need_retrain = True
seed = 123

In [4]:
import tensorflow as tf

general_params = {
    'phi_h': tf.keras.activations.linear,
    'phi_o': tf.keras.activations.linear,
}

train_params = {
    'learning_rate': 1e-1,
    'epochs_before_decay': 0.1,
    'epochs_count': 50,
    'learning_rate_decay': 1/3,
    'iterations_count': 300,
    'early_stopping_patience': 5,
}

In [5]:
def create_experiment_enviroment(model_name, experiment_name, mode):
    experiment_settings = dict()
    
    experiment_settings['model_name'] = model_name
    experiment_settings['experiment_name'] = experiment_name
    experiment_settings['mode'] = mode
    
    experiment_dir = path_join(EXPERIMENTS_DIR, experiment_name)
    make_directory(experiment_dir)
    experiment_settings['experiment_dir'] = experiment_dir
    
    tf_model_dir = path_join(experiment_dir, 'tf_model')
    make_directory(tf_model_dir)
    experiment_settings['tf_model_dir'] = tf_model_dir
    
    images_dir = path_join(experiment_dir, 'images')
    make_directory(images_dir)
    experiment_settings['images_dir'] = images_dir
    
    log_path = path_join(experiment_dir, 'log.log')
    logging.basicConfig(filename=log_path, level=logging.INFO)
    experiment_settings['log_path'] = log_path

    vensim_model_file = path_join(VENSIM_MODELS_DIR, '{}.mdl'.format(model_name))
    experiment_settings['vensim_model_file'] = vensim_model_file
    
    prn_model_dir = path_join(tf_model_dir, 'prn_model')
    nn_model_dir = path_join(tf_model_dir, 'base_nn_model')
    make_directory(prn_model_dir)
    make_directory(nn_model_dir)
    experiment_settings['prn_model_dir'] = prn_model_dir
    experiment_settings['nn_model_dir'] = nn_model_dir

    return experiment_settings
    
experiment_settings = create_experiment_enviroment(model_name, experiment_name, mode)

In [6]:
from definitions import path_join, DATA_DIR

dataset_dir = path_join(DATA_DIR, model_name)
dataset_file = path_join(dataset_dir, 'dataset.csv')

In [7]:
import pandas as pd

data = pd.read_csv(dataset_file)
dt = 0.03125
stopwords = ['TIME', 'sim_index']
fields = [column for column in data.columns if column not in stopwords]

In [8]:
from module.fd_model.vensim_fd_converter import create_unknown_model

FD = create_unknown_model(fields)
FD.dT = dt

print('dt: {}'.format(dt))

dt: 0.03125


In [9]:
fields = [level for level in FD.names_units_map.keys()]
fields

['Characteristic Time', 'Room Temperature', 'Teacup Temperature']

In [10]:
from sklearn.model_selection import train_test_split


def np_preproc_for_rnn3d(numpy_array, fields):
    x_groups = [group[1][fields].values[:-1] for group in numpy_array]
    y_groups = [group[1][fields].values[1:] for group in numpy_array]
    
    train_X, valid_X, train_y, valid_y = train_test_split(x_groups, y_groups, test_size=0.2, random_state=seed)
    
    train_X = np.concatenate(train_X, axis=0)
    valid_X = np.concatenate(valid_X, axis=0)
    
    train_y = np.concatenate(train_y, axis=0)
    valid_y = np.concatenate(valid_y, axis=0)

    return (train_X, train_y), (valid_X, valid_y)


def generate_train_data(df, fields):
    dataset = df[fields].values
    grouped = df.groupby(['sim_index'])[fields]
    
    return dataset, np_preproc_for_rnn3d(grouped, fields)

In [11]:
_, (prn_train, prn_valid) = generate_train_data(data, fields)

In [12]:
prn_train_X, prn_train_y = prn_train
prn_valid_X, prn_valid_y = prn_valid

In [13]:
prn_train_X.shape

(12000, 3)

In [14]:
import tensorflow as tf
from tensorflow.python.framework import ops

ops.reset_default_graph()

In [15]:
from arch.base_nn import BaseNN

predictor = BaseNN(prn_train_X.shape[1], prn_train_X.shape[1])

In [16]:
if need_retrain:
    predictor.train(prn_train_X, prn_train_y, (prn_valid_X, prn_valid_y), train_params, experiment_settings['nn_model_dir'])
    pass
nn_output = predictor.test(prn_valid_X, experiment_settings['nn_model_dir'])

Train on 12000 samples, validate on 3000 samples
Epoch 1/50
12000/12000 [==============================] - 1s 98us/sample - loss: 2026.9577 - mse: 2026.9585 - val_loss: 1113.9975 - val_mse: 1113.9972
Epoch 2/50
12000/12000 [==============================] - 1s 65us/sample - loss: 851.8382 - mse: 851.8379 - val_loss: 561.0353 - val_mse: 561.0352
Epoch 3/50
12000/12000 [==============================] - 1s 66us/sample - loss: 460.5712 - mse: 460.5715 - val_loss: 308.6769 - val_mse: 308.6770
Epoch 4/50
12000/12000 [==============================] - 1s 65us/sample - loss: 248.0350 - mse: 248.0351 - val_loss: 161.2519 - val_mse: 161.2519
Epoch 5/50
12000/12000 [==============================] - 1s 65us/sample - loss: 124.3901 - mse: 124.3901 - val_loss: 78.2769 - val_mse: 78.2769
Epoch 6/50
12000/12000 [==============================] - 1s 65us/sample - loss: 57.5635 - mse: 57.5635 - val_loss: 34.9408 - val_mse: 34.9408
Epoch 7/50
12000/12000 [==============================] - 1s 64us/sampl

In [17]:
nn_output[:5]

array([[10.000001, 69.99999 , 40.375206],
       [10.000001, 69.99999 , 40.750454],
       [10.000001, 69.99999 , 41.11583 ],
       [10.000001, 69.99999 , 41.47133 ],
       [10.000001, 69.99999 , 41.826828]], dtype=float32)

In [18]:
predictor.calculate_trainable_parameters()

12

In [19]:
def calculate_error(required_columns_data, output):
    output = np.array(output)
    error = sum(abs((output-required_columns_data)/required_columns_data))/required_columns_data.shape[0]

    return error

<h3>Train set error</h3>

In [27]:
initial_value = np.reshape(prn_train_X[0], [1, prn_train_y.shape[1]])
iterations_count = prn_train_X.shape[0]-1
train_prn_output = run_simulation(rnn_model, prn_model_dir, initial_value, iterations_count)
train_prn_error = calculate_error(train_y, train_prn_output)
train_prn_error

NameError: name 'run_simulation' is not defined

In [41]:
train_nn_output = predictor.test(train_X, nn_model_dir)
train_nn_error = calculate_error(train_y, train_nn_output)
train_nn_error

array([0.01026829, 0.2333064 , 0.00324267])

<h3>Test set error</h3>

In [41]:
predictor.test(train_X, nn_model_dir)
prn_error = calculate_error(valid_y, prn_output)
prn_error

ValueError: operands could not be broadcast together with shapes (384,6) (77,6) 

In [42]:
valid_nn_output = predictor.test(valid_X, nn_model_dir)
valid_nn_error = calculate_error(valid_y, valid_nn_output)
valid_nn_error

array([0.01008007, 0.23619833, 0.00327633])